# Toronto Part 3

## 1. Preparing

In [1]:
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

In [2]:
  
# import for plotting
import matplotlib.cm as cm
import matplotlib.colors as colors

# import K-means from clustering state
from sklearn.cluster import KMeans

# map rendering library
import folium

print('Libraries imported')

Libraries imported


In [3]:
!conda install -c conda-forge geopy --yes

WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.
Solving environment: done

## Package Plan ##

  environment location: /Users/jasonshieh/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h01d97ff_0         1.9 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.1 MB

The following NEW pack

## 2. convert an address into latitude and longtitude values

In [5]:
from geopy.geocoders import Nominatim

In [9]:
address = 'Toronto, ON'

geolocator = Nominatim (user_agent = 'Toronto')
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geographical coordinate of Toronto are {},{}.'.format(latitude_toronto, longtitude_toronto))

The geographical coordinate of Toronto are 43.653963,-79.387207.


In [12]:
Toronto_df = pd.read_csv('df_toronto.csv',index_col=0)
Toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M6L,Downsview,Downsview,43.713756,-79.490074
1,M4V,"Deer Park, Rathnelly, South Hill","Deer Park, Rathnelly, South Hill",43.686412,-79.400049
2,M5V,"CN Tower, King and Spadina, Railway Lands, Sou...","CN Tower, King and Spadina, Railway Lands, Sou...",43.628947,-79.394420
3,M2K,Bayview Village,Bayview Village,43.786947,-79.385975
4,M4A,Victoria Village,Victoria Village,43.725882,-79.315572


## 3. Map

In [13]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start = 10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighbourhood']):
    label = '{},{}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup=label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)
    
map_toronto
    

## 4. Getting More Infor

In [21]:
# Foursquare Credentials and version
CLIENT_ID = 'KWXN0YLCFRJTRPI2DUHNSOZWSWWJLONXV4JB2ZOX5ON1ZO25'
CLIENT_SECRET = 'EBCYLQNXTE0U0GIXD0GZXBTQ1SRP2H54CL14RNUK1Y2DHLVZ'
VERSION = '20190706'

print('Your credentials: ')
print('CLIENT_ID: '+CLIENT_ID)
print('CLIENT_SECRET: '+CLIENT_SECRET)

Your credentials: 
CLIENT_ID: KWXN0YLCFRJTRPI2DUHNSOZWSWWJLONXV4JB2ZOX5ON1ZO25
CLIENT_SECRET: EBCYLQNXTE0U0GIXD0GZXBTQ1SRP2H54CL14RNUK1Y2DHLVZ


In [22]:
# defining radius and limit of venues to get 
redius = 500
LIMIT = 100

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    venues_list =[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                                 'Neighbourhood Latitude', 
                                 'Neighbourhood Longitude', 
                                 'Venue',
                                 'Venue Latitude',
                                 'Venue Longitude',
                                 'Venue Category']
    return(nearby_venues)
        

In [26]:
toronto_venues = getNearbyVenues(names=Toronto_df['Neighbourhood'],
                                latitudes = Toronto_df['Latitude'],
                                 longitudes = Toronto_df['Longitude']
                                )

Downsview
Deer Park, Rathnelly, South Hill
CN Tower, King and Spadina, Railway Lands, South Niagara
Bayview Village
Victoria Village
Parkwoods
Henry Farm
Design Exchange, Toronto Dominion Centre
Cliffcrest, Cliffside
Morningside, West Hill
St. James Town
Weston
Dorset Park, Scarborough Town Centre, Wexford Heights
Woodbine Heights
Alderwood, Long Branch
Rouge, Malvern
Maryvale, Wexford
Riverdale
Upper Rouge
Exhibition Place, Parkdale Village
Humber Summit
The Kingsway
Humber Bay, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea
Keelesdale, Mount Dennis, Silverthorn
Willowdale West
Islington, Princess Gardens, West Deane Park
York Mills
India Bazaar
Newtonbrook, Willowdale
New Toronto
Emery, Humberlea
Thorncliffe Park
Woburn
Woodbine Gardens, Parkview Hill
Runnymede, Swansea
Rosedale
High Park
Parkdale, Roncesvalles
Mimico NW, The Queensway West
Highland Creek, Rouge Hill, Port Union
Leaside
Moore Park
Markland Wood
Scarborough Village
Agincourt
Chinatown, 

In [27]:
toronto_venues.head(10)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downsview,43.713756,-79.490074,Rustic Bakery,43.715414,-79.490300,Bakery
1,Downsview,43.713756,-79.490074,Maple leaf park,43.716188,-79.493531,Park
2,Downsview,43.713756,-79.490074,Mika's Trim,43.714068,-79.496113,Construction & Landscaping
3,"Deer Park, Rathnelly, South Hill",43.686412,-79.400049,LCBO,43.686991,-79.399238,Liquor Store
4,"Deer Park, Rathnelly, South Hill",43.686412,-79.400049,The Market By Longo’s,43.686711,-79.399536,Supermarket
5,"Deer Park, Rathnelly, South Hill",43.686412,-79.400049,Union Social Eatery,43.687895,-79.394916,American Restaurant
6,"Deer Park, Rathnelly, South Hill",43.686412,-79.400049,Daeco Sushi,43.687838,-79.395652,Sushi Restaurant
7,"Deer Park, Rathnelly, South Hill",43.686412,-79.400049,Mary Be Kitchen,43.687708,-79.395062,Restaurant
8,"Deer Park, Rathnelly, South Hill",43.686412,-79.400049,Starbucks,43.687101,-79.398612,Coffee Shop
9,"Deer Park, Rathnelly, South Hill",43.686412,-79.400049,Tim Hortons,43.687682,-79.396840,Coffee Shop


In [29]:
toronto_venues.shape

(1585, 7)

In [30]:
# How many venues were returned for each neighbourhood
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, Milliken",3,3,3,3,3,3
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Beaumond Heights, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
Bedford Park,22,22,22,22,22,22
Berczy Park,56,56,56,56,56,56
Birch Cliff,4,4,4,4,4,4


## 5. Analysing each Neighbourhood

In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix = '', prefix_sep = '')

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Downsview
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Downsview
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Downsview
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Deer Park, Rathnelly, South Hill"
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Deer Park, Rathnelly, South Hill"


In [32]:
toronto_onehot.shape

(1585, 251)

### 5.1 grouping rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

In [33]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.00,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Agincourt North, Milliken",0.00,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Alderwood, Long Branch",0.00,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Bathurst Manor, Wilson Heights",0.00,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Bayview Village,0.00,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Beaumond Heights, Jamestown, Mount Olive, Silv...",0.00,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Bedford Park,0.00,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.045455,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Berczy Park,0.00,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Birch Cliff,0.00,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,CFB Toronto,0.00,0.000000,0.5000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### 5.2  printing each neighbourhood along with the top 5 most common venues

In [35]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print('----'+hood+'----')
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending = False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
               venue  freq
0     Breakfast Spot  0.25
1     Clothing Store  0.25
2             Lounge  0.25
3       Skating Rink  0.25
4  Accessories Store  0.00


----Agincourt North, Milliken----
               venue  freq
0               Park  0.33
1         Playground  0.33
2                Gym  0.33
3  Accessories Store  0.00
4             Museum  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.22
1            Pool  0.11
2  Sandwich Place  0.11
3        Pharmacy  0.11
4             Pub  0.11


----Bathurst Manor, Wilson Heights----
                       venue  freq
0                Coffee Shop  0.11
1                Pizza Place  0.06
2  Middle Eastern Restaurant  0.06
3       Fast Food Restaurant  0.06
4        Fried Chicken Joint  0.06


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1  Japanese Restaurant  0.25
2   Chinese Restaurant  0.25
3                 Café  0.25
4        Movie Theater 

In [37]:
# function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0: num_top_venues]

In [38]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns = columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Clothing Store,Lounge,Skating Rink,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
1,"Agincourt North, Milliken",Playground,Park,Gym,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
2,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Skating Rink,Pharmacy,Pool,Pub,Sandwich Place,Gym,Airport Terminal,American Restaurant
3,"Bathurst Manor, Wilson Heights",Coffee Shop,Restaurant,Diner,Bank,Deli / Bodega,Supermarket,Middle Eastern Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Sandwich Place
4,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Yoga Studio


### 5.3 Cluster Neighbourhoods

In [40]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering 
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_


array([0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 0, 2, 0, 0, 0, 0, 3], dtype=int32)

In [ ]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood

In [44]:


# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = Toronto_df

# merge toronto_grouped with toronto_data to add latitude/longtitude for each neighborhod
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M6L,Downsview,Downsview,43.713756,-79.490074,2.0,Park,Bakery,Construction & Landscaping,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Yoga Studio
1,M4V,"Deer Park, Rathnelly, South Hill","Deer Park, Rathnelly, South Hill",43.686412,-79.400049,0.0,Coffee Shop,Pub,Bagel Shop,Medical Center,Restaurant,Supermarket,Fried Chicken Joint,Sports Bar,American Restaurant,Sushi Restaurant
2,M5V,"CN Tower, King and Spadina, Railway Lands, Sou...","CN Tower, King and Spadina, Railway Lands, Sou...",43.628947,-79.394420,0.0,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Bar,Plane,Coffee Shop,Sculpture Garden,Boat or Ferry,Boutique
3,M2K,Bayview Village,Bayview Village,43.786947,-79.385975,0.0,Café,Japanese Restaurant,Bank,Chinese Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Yoga Studio
4,M4A,Victoria Village,Victoria Village,43.725882,-79.315572,0.0,Pizza Place,French Restaurant,Coffee Shop,Portuguese Restaurant,Intersection,Hockey Arena,Doner Restaurant,Discount Store,Dive Bar,Dog Run


In [45]:
toronto_merged = toronto_merged.dropna()

In [55]:
# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start = 11)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map 
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'],
                                 toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi)+'Cluster'+str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color =rainbow[1],
        fill = True,
        fill_color = rainbow[1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

### 5.4 Examine Clusters 

In [57]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Deer Park, Rathnelly, South Hill",0.0,Coffee Shop,Pub,Bagel Shop,Medical Center,Restaurant,Supermarket,Fried Chicken Joint,Sports Bar,American Restaurant,Sushi Restaurant
2,"CN Tower, King and Spadina, Railway Lands, Sou...",0.0,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Bar,Plane,Coffee Shop,Sculpture Garden,Boat or Ferry,Boutique
3,Bayview Village,0.0,Café,Japanese Restaurant,Bank,Chinese Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Yoga Studio
4,Victoria Village,0.0,Pizza Place,French Restaurant,Coffee Shop,Portuguese Restaurant,Intersection,Hockey Arena,Doner Restaurant,Discount Store,Dive Bar,Dog Run
5,Parkwoods,0.0,Fast Food Restaurant,Park,Food & Drink Shop,Bus Stop,Diner,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
6,Henry Farm,0.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Restaurant,Bakery,Cosmetics Shop,Food Court,Japanese Restaurant,Candy Store
7,"Design Exchange, Toronto Dominion Centre",0.0,Coffee Shop,Café,Hotel,Italian Restaurant,Restaurant,Bar,Gym,American Restaurant,Gastropub,Deli / Bodega
8,"Cliffcrest, Cliffside",0.0,American Restaurant,Motel,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
9,"Morningside, West Hill",0.0,Spa,Pizza Place,Intersection,Breakfast Spot,Medical Center,Electronics Store,Rental Car Location,Mexican Restaurant,Donut Shop,Dive Bar
10,St. James Town,0.0,Coffee Shop,Hotel,Restaurant,Café,Cosmetics Shop,Breakfast Spot,Bakery,Gastropub,Italian Restaurant,Creperie


In [58]:
# cluster 2
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Humber Summit,1.0,Empanada Restaurant,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [59]:
# cluster 3
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downsview,2.0,Park,Bakery,Construction & Landscaping,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Yoga Studio
11,Weston,2.0,Park,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
21,The Kingsway,2.0,Park,River,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
22,"Humber Bay, Mimico NE, Old Mill South, The Que...",2.0,Park,Pool,Baseball Field,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
35,Rosedale,2.0,Park,Playground,Building,Trail,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Dumpling Restaurant
52,CFB Toronto,2.0,Park,Airport,Yoga Studio,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
54,East Toronto,2.0,Park,Pizza Place,Coffee Shop,Convenience Store,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
73,"Agincourt North, Milliken",2.0,Playground,Park,Gym,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [60]:
# cluster 4
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,York Mills,3.0,Cafeteria,Yoga Studio,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [61]:
# cluster 5
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,"Highland Creek, Rouge Hill, Port Union",4.0,Bar,Yoga Studio,Filipino Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
